In [1]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.8 MB/s eta 0:00:00


In [2]:
!pip install streamlit joblib

In [3]:
!apt-get update -qq
!apt-get install openjdk-8-jdk -y
!pip install -q pyspark


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java libatk-wrapper-java-jni libfontenc1
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libice-dev librsvg2-common
  libsm-dev libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-8-jdk-headless openjdk-8-jre
  openjdk-8-jre-headless x11-utils
Suggested packages:
  gvfs libice-doc libsm-doc libxt-doc openjdk-8-demo openjdk-8-source visualvm libnss-mdns
  fonts-nanum fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei
  fonts-indic mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java libatk-w

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"


In [9]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
from pyspark.ml.regression import GBTRegressionModel
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline, PipelineModel  # Import Pipeline và PipelineModel
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import SparkSession

# Khởi tạo SparkSession
spark = SparkSession.builder.appName("FuelConsumptionPrediction").getOrCreate()

# Tải mô hình đã huấn luyện dưới dạng PipelineModel
model_path = "/content/drive/MyDrive/datasets/GBTModel"  # Đường dẫn đến mô hình đã lưu
loaded_pipeline_model = PipelineModel.load(model_path)  # Tải mô hình PipelineModel

# Trích xuất mô hình GBTRegressionModel từ pipeline
loaded_gbt_model = loaded_pipeline_model.stages[-1]  # GBTRegressionModel là bước cuối cùng trong pipeline

# Tạo các bước trong pipeline như trong quá trình huấn luyện
indexer_Ft = StringIndexer(inputCol="Ft", outputCol="Ft_index")
indexer_Fm = StringIndexer(inputCol="Fm", outputCol="Fm_index")
encoder_Ft = OneHotEncoder(inputCol="Ft_index", outputCol="Ft_encoded")
encoder_Fm = OneHotEncoder(inputCol="Fm_index", outputCol="Fm_encoded")
assembler = VectorAssembler(
    inputCols=["m (kg)", "Ft_encoded", "Fm_encoded", "ec (cm3)", "ep (KW)", "Electric range (km)"],
    outputCol="assembled_features"
)
scaler = StandardScaler(inputCol="assembled_features", outputCol="features")

# Tạo pipeline với các bước trên
pipeline = Pipeline(stages=[indexer_Ft, indexer_Fm, encoder_Ft, encoder_Fm, assembler, scaler])

# Tạo giao diện Streamlit
st.title("Fuel Consumption Prediction using Gradient Boosting Regressor")
st.write("Dự đoán mức tiêu thụ nhiên liệu (Fuel consumption) dựa trên các đặc trưng như trọng lượng, kiểu động cơ, và các đặc tính khác.")

# Nhận đầu vào từ người dùng
m = st.number_input("Trọng lượng (m) (kg):", min_value=0, max_value=10000, value=1000)
Ft = st.selectbox("Kiểu động cơ (Ft):", ['petrol', 'ng', 'diesel-electric', 'e85', 'lpg',  'diesel',  'lgp'])  # Điều chỉnh theo các giá trị thực tế trong dữ liệu
Fm = st.selectbox("Loại động cơ (Fm):", ['F', 'B', 'P', 'M', 'H', 'F', 'B', 'M', 'P', 'M', 'H'])  # Điều chỉnh theo các giá trị thực tế trong dữ liệu
ec = st.number_input("Dung tích động cơ (ec) (cm3):", min_value=0, max_value=10000, value=2000)
ep = st.number_input("Công suất động cơ (ep) (KW):", min_value=0, max_value=1000, value=100)
electric_range = st.number_input("Phạm vi di chuyển (Electric range) (km):", min_value=0, max_value=1000, value=100)

# Khi người dùng nhấn nút "Dự đoán"
if st.button("Dự đoán"):
    # Tạo DataFrame từ đầu vào người dùng
    input_data = pd.DataFrame({
        "m (kg)": [m],
        "Ft": [Ft],
        "Fm": [Fm],
        "ec (cm3)": [ec],
        "ep (KW)": [ep],
        "Electric range (km)": [electric_range]
    })

    # Chuyển DataFrame thành DataFrame Spark
    spark_df = spark.createDataFrame(input_data)

    # Tiền xử lý dữ liệu bằng pipeline
    processed_data = loaded_pipeline_model.transform(spark_df)

    # Loại bỏ cột 'prediction' nếu đã tồn tại
    processed_data = processed_data.drop('prediction')

    # Dự đoán Fuel consumption
    prediction = loaded_gbt_model.transform(processed_data)

    # Hiển thị kết quả dự đoán
    prediction_value = prediction.select("prediction").collect()[0]["prediction"]
    st.write(f"Dự đoán mức tiêu thụ nhiên liệu là: {prediction_value:.2f} (L/100km)")



Overwriting app.py


In [6]:
!wget -q -O - ipv4.icanhazip.com

35.238.94.27


In [7]:
!streamlit run app.py & npx localtunnel --port 8501 --no-auth

⠙⠹

⠸⠼⠴⠦⠧Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) 
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.238.94.27:8501

  Stopping...
^C


In [ ]:
!streamlit run app.py





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.87.191:8501

  Stopping...
  Stopping...


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.238.94.27:8501

⠼⠴your url is: https://cold-dolls-smash.loca.lt
2024-12-04 07:02:09.668 Uncaught exception GET /_stcore/stream (127.0.0.1)
HTTPServerRequest(protocol='http', host='cold-dolls-smash.loca.lt', method='GET', uri='/_stcore/stream', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tornado/websocket.py", line 937, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/streamlit/web/server/browser_websocket_handler.py", line 126, in open
    self._session_id = self._runtime.connect_session(
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/runtime.py", line 384, in connect_session
    session_id = self._session_mgr.connect_session(
  File 